In [ ]:
import os
from sys import path
import pandas as pd
from sqlalchemy import create_engine
path.append('python-files')
from nasdaq_api_call import wiki_api_call, zacks_fc_api_call
pd.set_option('display.max_columns', None)


In [ ]:
#Read source
source_data_path= os.path.join('data', 'post_Y2K_tickers_for_webscraping.csv')
source_data = pd.read_csv(source_data_path)

#format date for functions
def format_date_col_string(col):
    source_data[col] = [str(d)[:10] for d in source_data[col]]

format_date_col_string('start_date')
format_date_col_string('end_date')

print(source_data.dtypes)

In [ ]:
#Create DB Connection
conn = os.getenv('STOCK_DB_CONN')
engine = create_engine(conn)

zipper = list(zip(source_data['ticker'], source_data['start_date'], source_data['end_date']))

In [ ]:


#Pull WIKI historical daily prices
for t, s, e in zipper:
    df = wiki_api_call(t, s, e)
    if len(df) > 0:
        df.to_sql(name = 'wiki_prices', con = conn, schema = 'NASDAQ', if_exists= 'append', index = False, method = 'multi')

In [ ]:
#columns to include in response from ZACKS/FC api call
zacks_columns = [
    'm_ticker', #primary_key
    'ticker', 'comp_name', 'currency_code', 'per_end_date', 'per_fisc_year', 'per_cal_year', 'filing_date',
    'zacks_sector_code', 'zacks_x_ind_code', 
    'bus_city', 'bus_state_name', 'bus_post_code', 'country_name',
    'tot_revnu', #millions
    'gross_profit', #millions
    'tot_oper_exp', #millions
    'basic_net_eps', 
    'comm_shares_out', 'comm_stock_div_paid',
    'incr_decr_cash', 
]

In [ ]:

for t, s, e in zipper:
    df = zacks_fc_api_call(t, s, e, columns = zacks_columns)
    df = df[zacks_columns]
    if len(df) > 0:
        df.to_sql(name = 'zacks_fc', con = conn, schema = 'NASDAQ', if_exists= 'append', index = False, method = 'multi')
        